# OSTk Cross-Platform Validation Against GMAT/Orekit (Mission Sequence Scenarios 1-3)

This tutorial demonstrates how to compare OSTk numerical orbit propagation with other tools like Orekit or GMAT. This example will be for a mission sequence that consists of a coasting phases.

In [9]:
import numpy as np
import pandas as pd
import csv
import os
import plotly.graph_objs as go
from ostk.astrodynamics.trajectory.state import NumericalSolver
from ostk.astrodynamics.display import create_2d_map, create_3d_globe
from ostk.astrodynamics.flight.system import SatelliteSystem, PropulsionSystem
from ostk.astrodynamics import Dynamics
from ostk.astrodynamics.trajectory import LocalOrbitalFrameFactory
from ostk.astrodynamics.trajectory import LocalOrbitalFrameDirection
from ostk.astrodynamics.trajectory.state import CoordinatesSubset
from ostk.astrodynamics.trajectory.state.coordinates_subset import CartesianPosition
from ostk.astrodynamics.trajectory.state.coordinates_subset import CartesianVelocity
from ostk.astrodynamics.trajectory import StateBuilder
from ostk.astrodynamics.dynamics import (
    AtmosphericDrag,
    CentralBodyGravity,
    PositionDerivative,
    ThirdBodyGravity,
    Thruster,
)
from ostk.astrodynamics.guidance_law import ConstantThrust
from ostk.astrodynamics.trajectory import Orbit, Propagator, State
from ostk.astrodynamics.trajectory.orbit.models import Propagated
from ostk.core.filesystem import Directory
from ostk.mathematics.geometry.d3.objects import Composite, Cuboid, Point
from ostk.physics import Environment
from ostk.physics.coordinate import Frame, Position, Velocity
from ostk.physics.coordinate.spherical import LLA
from ostk.physics.environment.atmospheric import Earth as EarthAtmosphericModel
from ostk.physics.environment.gravitational import Earth as EarthGravitationalModel
from ostk.physics.environment.magnetic import Earth as EarthMagneticModel
from ostk.physics.environment.objects.celestial_bodies import Earth, Moon, Sun
from ostk.physics.time import DateTime, Duration, Instant, Interval, Scale
from ostk.physics.units import Mass

from ostk.mathematics.geometry.d3.objects import Cuboid
from ostk.mathematics.geometry.d3.objects import Composite
from ostk.mathematics.geometry.d3.objects import Point

## Set up Comparison Files

Define array inputs that can be changed to include/exclude gmat/orekit result comparisons

In [64]:
filenames = ["gmat_astrodynamics/scenario001-mission-sequence.csv", "orekit_astrodynamics/scenario001-mission-sequence.csv"]
# filenames = ["orekit/scenario001-mission-sequence.csv"]
# filenames = ["gmat/scenario003-mission-sequence.csv"]
legend_name_list = ["GMAT","OREKIT"]

multiplication_factors = [np.array([1.0e3, 1.0e3, 1.0e3]), np.array([1.0, 1.0, 1.0])]

## Setup Comparison Scenario in OSTk

Define the physical `Environment`

In [65]:
earth = Earth.from_models(
    EarthGravitationalModel(
        EarthGravitationalModel.Type.EGM96, Directory.undefined(), 0, 0
    ),
    EarthMagneticModel(EarthMagneticModel.Type.Undefined),
    EarthAtmosphericModel(EarthAtmosphericModel.Type.Undefined),
)
environment = Environment(Instant.J2000(), [earth])

Define the `SatelliteSystem` 

In [66]:
mass = Mass(100.0, Mass.Unit.Kilogram)
satellite_geometry = Composite(
    Cuboid(
        Point(0.0, 0.0, 0.0),
        np.eye(3).tolist(),
        [1.0, 0.0, 0.0],
    )
)
inertia_tensor = np.identity(3)
surface_area = 1.0
drag_coefficient = 2.2

satellite_system = SatelliteSystem(
    mass,
    satellite_geometry,
    inertia_tensor,
    surface_area,
    drag_coefficient,
)

Define the initial `State` of the `SatelliteSystem`

In [67]:
start_instant = Instant.date_time(DateTime.parse("2023-01-01T00:00:00.000"), Scale.UTC)
initial_state = State(
    start_instant,
    Position.meters(
        np.array([-4283387.412456233, -4451426.776125101, -2967617.850750065]),
        Frame.GCRF(),
    ),
    Velocity.meters_per_second(
        np.array([4948.074939732174, -957.3429532772124, -5721.173027553034]),
        Frame.GCRF(),
    ),
)

In [68]:
state_builder = StateBuilder(
    frame=Frame.GCRF(),
    coordinates_subsets=[
        CartesianPosition.default(),
        CartesianVelocity.default(),
        CoordinatesSubset.mass(),
        CoordinatesSubset.surface_area(),
        CoordinatesSubset.drag_coefficient(),
    ],
)

coordinates = [
    *initial_state.get_coordinates().tolist(),
    mass.in_kilograms(),
    satellite_system.get_cross_sectional_surface_area(),
    satellite_system.get_drag_coefficient(),
]
start_state = state_builder.build(initial_state.get_instant(), coordinates)

Define the Dynamics to consider for Propagation

In [69]:
dynamics = Dynamics.from_environment(environment)

Define a `NumericalSolver` 

In [70]:
numerical_solver = NumericalSolver(
    NumericalSolver.LogType.NoLog,
    NumericalSolver.StepperType.RungeKutta4,
    30.0,
    1.0e-1,
    1.0e-1,
)

Define a `Propagator`

In [71]:
propagator = Propagator(numerical_solver, dynamics)

### Propagate with OSTk

Define the propagation `instants` of interest

In [72]:
instants = Interval.closed(
    start_state.get_instant(), start_state.get_instant() + Duration.seconds(86400.0)
).generate_grid(Duration.seconds(120.0))

Now that everything is set up, we can calculate the state arrays from the desired time instant grid

In [73]:
ostk_states = propagator.calculate_states_at(start_state, instants)

---

## Process Cross Platform Results 

Read in reference data from CSV file for GMAT and Orekit

In [74]:
all_comparison_states = []

for ind, filename in enumerate(filenames):
    with open(f"{os.getcwd()}/data/{filename}") as csvfile:
        reader = csv.DictReader(csvfile)
        comparison_states = []
        comparison_instants = []
        for row in reader:
            instant_iter = start_instant + Duration.seconds(
                float(row[reader.fieldnames[0]])
            )
            position_iter = Position.meters(
                [
                    row[reader.fieldnames[1]],
                    row[reader.fieldnames[2]],
                    row[reader.fieldnames[3]],
                ],
                Frame.GCRF(),
            )
            velocity_iter = Velocity.meters_per_second(
                [
                    row[reader.fieldnames[4]],
                    row[reader.fieldnames[5]],
                    row[reader.fieldnames[6]],
                ],
                Frame.GCRF(),
            )

            position_iter_m = np.multiply(
                position_iter.get_coordinates(), multiplication_factors[ind]
            )
            velocity_iter_ms = np.multiply(
                velocity_iter.get_coordinates(), multiplication_factors[ind]
            )

            comparison_instants.append(instant_iter)
            comparison_states.append(
                State(
                    instant_iter,
                    Position.meters(position_iter_m, Frame.GCRF()),
                    Velocity.meters_per_second(velocity_iter_ms, Frame.GCRF()),
                )
            )
    all_comparison_states.append(comparison_states)

#### Trajectory RMS error vs GMAT run in the GCRF frame 

In [75]:
def to_dataframe_RMS(state_ind, list_ind):
    return [
        repr(all_comparison_states[list_ind][state_ind].get_instant()),
        float(
            (
                all_comparison_states[list_ind][state_ind].get_instant()
                - all_comparison_states[list_ind][0].get_instant()
            ).in_seconds()
        ),
        (
            np.linalg.norm(
                (
                    all_comparison_states[list_ind][state_ind]
                    .get_position()
                    .get_coordinates()
                    - ostk_states[state_ind].get_position().get_coordinates()
                )
            )
        ),
        (
            np.linalg.norm(
                (
                    all_comparison_states[list_ind][state_ind]
                    .get_velocity()
                    .get_coordinates()
                    - ostk_states[state_ind].get_velocity().get_coordinates()
                )
            )
        ),
    ]

In [76]:
ostk_states_compared = [
    [
        to_dataframe_RMS(state_ind, list_ind)
        for state_ind in range(0, len(all_comparison_states[list_ind]))
    ]
    for list_ind in range(0, len(all_comparison_states))
]

In [77]:
ostk_states_compared_df = [
    pd.DataFrame(
        ostk_states_compared[list_ind],
        columns=["$Time^{UTC}$", "Elapsed secs", "${\delta}x$", "${\delta}v$"],
    )
    for list_ind in range(0, len(all_comparison_states))
]

In [78]:
[
    ostk_states_compared_df[list_ind].head()
    for list_ind in range(0, len(all_comparison_states))
]

[                            $Time^{UTC}$  Elapsed secs   ${\delta}x$  \
 0              2023-01-01 00:00:00 [UTC]           0.0  2.281265e-09   
 1  2023-01-01 00:02:00.000.000.076 [UTC]         120.0  4.594900e-05   
 2  2023-01-01 00:04:00.000.000.153 [UTC]         240.0  1.849503e-04   
 3  2023-01-01 00:06:00.000.000.230 [UTC]         360.0  4.209760e-04   
 4  2023-01-01 00:08:00.000.000.307 [UTC]         480.0  7.601324e-04   
 
     ${\delta}v$  
 0  2.572439e-12  
 1  7.677398e-07  
 2  1.558837e-06  
 3  2.395237e-06  
 4  3.296025e-06  ,
                 $Time^{UTC}$  Elapsed secs   ${\delta}x$   ${\delta}v$
 0  2023-01-01 00:00:00 [UTC]           0.0  9.313226e-10  1.905739e-12
 1  2023-01-01 00:02:00 [UTC]         120.0  9.475397e-03  1.013569e-05
 2  2023-01-01 00:04:00 [UTC]         240.0  1.885044e-02  2.119818e-05
 3  2023-01-01 00:06:00 [UTC]         360.0  2.802418e-02  3.396208e-05
 4  2023-01-01 00:08:00 [UTC]         480.0  3.701064e-02  4.900575e-05]

In [79]:
[
    ostk_states_compared_df[list_ind].tail()
    for list_ind in range(0, len(all_comparison_states))
]

[                              $Time^{UTC}$  Elapsed secs  ${\delta}x$  \
 716  2023-01-01 23:52:00.000.055.013 [UTC]  85920.000055     0.973292   
 717  2023-01-01 23:54:00.000.055.090 [UTC]  86040.000055     0.974629   
 718  2023-01-01 23:56:00.000.055.166 [UTC]  86160.000055     0.976166   
 719  2023-01-01 23:58:00.000.055.243 [UTC]  86280.000055     0.977899   
 720  2023-01-02 00:00:00.000.055.320 [UTC]  86400.000055     0.979823   
 
      ${\delta}v$  
 716     0.001089  
 717     0.001090  
 718     0.001092  
 719     0.001094  
 720     0.001096  ,
                   $Time^{UTC}$  Elapsed secs  ${\delta}x$  ${\delta}v$
 716  2023-01-01 23:52:00 [UTC]       85920.0    73.785103     0.082191
 717  2023-01-01 23:54:00 [UTC]       86040.0    73.959771     0.082395
 718  2023-01-01 23:56:00 [UTC]       86160.0    74.139079     0.082604
 719  2023-01-01 23:58:00 [UTC]       86280.0    74.323035     0.082817
 720  2023-01-02 00:00:00 [UTC]       86400.0    74.511568     0.083035]

# Validation Plots 

Plot position error 

In [80]:
orbit_df_RMS_position_list = [
    ostk_states_compared_df[list_ind][["Elapsed secs", "${\delta}x$"]]
    for list_ind in range(0, len(all_comparison_states))
]

figure = go.Figure()
figure.update_layout(title=f"Position Difference RMS", showlegend=True, height=1000)
figure.update_xaxes(title_text="Time Elapsed (s)")
figure.update_yaxes(title_text="Position Difference (m)")

for list_ind, orbit_df_RMS_position in enumerate(orbit_df_RMS_position_list):
    figure.add_trace(
        go.Scatter(
            x=orbit_df_RMS_position["Elapsed secs"],
            y=orbit_df_RMS_position["${\delta}x$"],
            name=legend_name_list[list_ind],
            mode="lines",
        )
    )

figure.show()

Plot velocity error

In [63]:
orbit_df_RMS_velocity_list = [
    ostk_states_compared_df[list_ind][["Elapsed secs", "${\delta}v$"]]
    for list_ind in range(0, len(all_comparison_states))
]

figure = go.Figure()
figure.update_layout(title=f"Velocity Difference RMS", showlegend=True, height=1000)
figure.update_xaxes(title_text="Time Elapsed (s)")
figure.update_yaxes(title_text="Velocity Difference (m/s)")

for list_ind, orbit_df_RMS_velocity in enumerate(orbit_df_RMS_velocity_list):
    figure.add_trace(
        go.Scatter(
            x=orbit_df_RMS_velocity["Elapsed secs"],
            y=orbit_df_RMS_velocity["${\delta}v$"],
            name=legend_name_list[list_ind],
            mode="lines",
        )
    )

figure.show()